In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [98]:
prgeng = pd.read_csv("prgeng.csv")

In [99]:
prgengprgeng.head()

,Unnamed: 0,age,cit,educ,engl,occ,birth,sex,wageinc,wkswrkd,yrentry,powspuma
0,1,50.300816,1,13,0,102,6,2,75000,52,0,6010
1,2,41.101390,1,9,0,101,6,1,12300,20,0,0
2,3,24.673740,1,9,0,102,6,2,15400,52,0,6010
3,4,50.199512,1,11,0,100,8,1,0,52,0,0
4,5,51.181124,1,11,0,100,6,2,160,1,0,6010


In [102]:
prgeng.drop(["Unnamed: 0"], inplace= True, axis= 1)
prgeng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20090 entries, 0 to 20089
Data columns (total 11 columns):
age         20090 non-null float64
cit         20090 non-null int64
educ        20090 non-null int64
engl        20090 non-null int64
occ         20090 non-null int64
birth       20090 non-null int64
sex         20090 non-null int64
wageinc     20090 non-null int64
wkswrkd     20090 non-null int64
yrentry     20090 non-null int64
powspuma    20090 non-null int64
dtypes: float64(1), int64(10)
memory usage: 1.7 MB


In [103]:
prgeng.describe()

,age,cit,educ,engl,occ,birth,sex,wageinc,wkswrkd,yrentry,powspuma
count,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000,20090.000000
mean,39.551056,2.356048,12.639174,5.062369,108.520707,98.711996,1.244301,60341.348432,45.158785,774.215580,5590.435391
std,11.222363,1.734853,1.638846,7.337958,15.313064,108.014591,0.429682,48777.287353,14.600989,968.526747,3065.687950
min,16.005762,1.000000,1.000000,0.000000,100.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.811983,1.000000,12.000000,0.000000,101.000000,6.000000,1.000000,31900.000000,48.000000,0.000000,6130.000000
50%,38.215150,1.000000,13.000000,0.000000,102.000000,36.000000,1.000000,58000.000000,52.000000,0.000000,6160.000000
75%,46.951904,4.000000,14.000000,10.000000,102.000000,210.000000,1.000000,80000.000000,52.000000,1982.000000,6309.000000
max,93.734037,5.000000,16.000000,41.000000,141.000000,528.000000,2.000000,325000.000000,52.000000,2000.000000,53080.000000


In [104]:
## Adding ms and phd column regarding education year
## Adding fem column whether individual is female or male. In original data set, code 1 is for male and code 2 is for female.
prgeng["ms"]= np.where(prgeng["educ"]== 14, 1, 0)
prgeng["phd"]= np.where(prgeng["educ"]== 16, 1, 0)
prgeng["fem"]= np.where(prgeng["sex"] == 2, 1, 0) 

In [105]:
prgeng.head()

,age,cit,educ,engl,occ,birth,sex,wageinc,wkswrkd,yrentry,powspuma,ms,phd,fem
0,50.300816,1,13,0,102,6,2,75000,52,0,6010,0,0,1
1,41.101390,1,9,0,101,6,1,12300,20,0,0,0,0,0
2,24.673740,1,9,0,102,6,2,15400,52,0,6010,0,0,1
3,50.199512,1,11,0,100,8,1,0,52,0,0,0,0,0
4,51.181124,1,11,0,100,6,2,160,1,0,6010,0,0,1


In [106]:
## Adding interaction parameters which are "age times gender" and "age times age" (square of age)
prgeng["age_squared"]= prgeng["age"]**2
prgeng["age&gender"]= prgeng["age"] * prgeng["fem"]

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

In [108]:
## In the linear model, we will include age, age_squared, wkswrkd, ms, phd, fem and age&gender to calculate wageinc
## Though the model is quadratic, parameters can be used in the linear model.
X= prgeng[["age", "age_squared", "wkswrkd", "ms", "phd", "fem", "age&gender"]]
y= prgeng["wageinc"]

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=101)

In [110]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
predictions = linear_model.predict(X_test)

In [111]:
# Getting correlation coefficient for features
coeff = pd.DataFrame(X.columns)
coeff.columns = ['Features']
coeff["Coefficient Estimate"] = pd.Series(linear_model.coef_)
coeff

,Features,Coefficient Estimate
0,age,4104.906321
1,age_squared,-41.895243
2,wkswrkd,1192.006165
3,ms,14990.688268
4,phd,25208.258391
5,fem,-219.286596
6,age&gender,-286.799285


In [112]:
## Getting intercept
linear_model.intercept_

-86317.33570010035

In [113]:
print("Linear model to predict wage increase:", float(linear_model.intercept_), "+",
      coeff.iloc[0][0], " * " ,coeff.iloc[0][1], "+ ",
      coeff.iloc[1][0], " * " ,coeff.iloc[1][1], "+ ",
      coeff.iloc[2][0], " * " ,coeff.iloc[2][1], "+ ",
      coeff.iloc[3][0], " * " ,coeff.iloc[3][1], "+ ",
      coeff.iloc[4][0], " * " ,coeff.iloc[4][1], "+ ",
      coeff.iloc[5][0], " * " ,coeff.iloc[5][1], "+ ",
      coeff.iloc[6][0], " * " ,coeff.iloc[6][1])   

Linear model to predict wage increase: -86317.33570010035 + age  *  4104.906320568408 +  age_squared  *  -41.895243417366835 +  wkswrkd  *  1192.006164618216 +  ms  *  14990.688268494188 +  phd  *  25208.258390572053 +  fem  *  -219.28659570118884 +  age&gender  *  -286.7992852356054


In [114]:
## Predicting wage increase of a 32 years-old female who has master degree and works 40 hours a week
print("Wage increase prediction of a 32 years-old female who has master degree:", float(linear_model.intercept_), "+",
      coeff.iloc[0][1], " * " ,32, "+ ",
      coeff.iloc[1][1], " * " ,32*32, "+ ",
      coeff.iloc[2][1], " * " ,40, "+ ",
      coeff.iloc[3][1], " * " ,1, "+ ",
      coeff.iloc[4][1], " * " ,0, "+ ",
      coeff.iloc[5][1], " * " ,1, "+ ",
      coeff.iloc[6][1], " * " , 32*1,)   

Wage increase prediction of a 32 years-old female who has master degree: -86317.33570010035 + 4104.906320568408  *  32 +  -41.895243417366835  *  1024 +  1192.006164618216  *  40 +  14990.688268494188  *  1 +  25208.258390572053  *  0 +  -219.28659570118884  *  1 +  -286.7992852356054  *  32


In [115]:
## Predicting wage increase of a 32 years-old female who has master degree and works 40 hours a week
print("Wage increase of a 32 years-old female who has master degree: $",float(linear_model.intercept_) + 
      coeff.iloc[0][1] * 32 +
      coeff.iloc[1][1] * 32*32 +
      coeff.iloc[2][1] * 40 + 
      coeff.iloc[3][1] * 1 +
      coeff.iloc[4][1] * 0 +
      coeff.iloc[5][1] * 1 +
      coeff.iloc[6][1] * 32*1)   

Wage increase of a 32 years-old female who has master degree: $ 55413.00842868733


In [67]:
## Another model, which does not take into account age&gender as a parameter.
X= prgeng[["age", "age_squared", "wkswrkd", "ms", "phd", "fem"]]
y= prgeng["wageinc"]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=101)

In [72]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
predictions = linear_model.predict(X_test)

In [77]:
# Getting correlation coefficient for features
coeff = pd.DataFrame(X.columns)
coeff.columns = ['Features']
coeff["Coefficient Estimate"] = pd.Series(linear_model.coef_)
coeff

,Features,Coefficient Estimate
0,age,3989.720391
1,age_squared,-41.183698
2,wkswrkd,1193.372365
3,ms,15127.536618
4,phd,25380.972525
5,fem,-11479.485660


In [78]:
## Getting intercept
linear_model.intercept_

-83067.49600814658

In [79]:
print("Linear model to predict wage increase:", float(linear_model.intercept_), "+",
      coeff.iloc[0][0], " * " ,coeff.iloc[0][1], "+ ",
      coeff.iloc[1][0], " * " ,coeff.iloc[1][1], "+ ",
      coeff.iloc[2][0], " * " ,coeff.iloc[2][1], "+ ",
      coeff.iloc[3][0], " * " ,coeff.iloc[3][1], "+ ",
      coeff.iloc[4][0], " * " ,coeff.iloc[4][1], "+ ",
      coeff.iloc[5][0], " * " ,coeff.iloc[5][1])  

Linear model to predict wage increase: -83067.49600814658 + age  *  3989.7203905496676 +  age_squared  *  -41.18369785124812 +  wkswrkd  *  1193.37236470181 +  ms  *  15127.53661843803 +  phd  *  25380.972525256184 +  fem  *  -11479.485660216758


In [81]:
import statsmodels.formula.api as smf

In [121]:
df= prgeng[["wageinc","age", "age_squared", "wkswrkd", "ms", "phd", "fem"]]
df.head()

,wageinc,age,age_squared,wkswrkd,ms,phd,fem
0,75000,50.300816,2530.172097,52,0,0,1
1,12300,41.101390,1689.324291,20,0,0,0
2,15400,24.673740,608.793464,52,0,0,1
3,0,50.199512,2519.991010,52,0,0,0
4,160,51.181124,2619.507411,1,0,0,1


In [123]:
## Creating a fitted model
lm1 = smf.ols(formula='wageinc ~ age+age_squared+wkswrkd+ms+phd+fem', data=df).fit()

In [146]:
# Coefficients
coefficients = pd.DataFrame(lm1.params)
coefficients.columns = ["Coefficients"]
coefficients

,Coefficients
Intercept,-81136.701681
age,3900.352215
age_squared,-40.328893
wkswrkd,1196.389763
ms,15431.073935
phd,23183.973204
fem,-11484.490055


In [144]:
## 95% Confidence Interval
CI = pd.DataFrame(lm1.conf_int())
CI.columns = ["Lower Confidence Interval", "Upper Confidence Interval"]
CI

,Lower Confidence Interval,Upper Confidence Interval
Intercept,-87575.081126,-74698.322236
age,3569.542149,4231.162280
age_squared,-44.150978,-36.506808
wkswrkd,1153.477198,1239.302327
ms,13982.970854,16879.177015
phd,19995.513332,26372.433077
fem,-12866.934849,-10102.045261


In [ ]:
print("Linear model to predict density:", float(linear_model.intercept_), "+",
      coeff.iloc[0][0], " * " ,coeff.iloc[0][1], "+ ",
      coeff.iloc[1][0], " * " ,coeff.iloc[1][1], "+ ",
      coeff.iloc[2][0], " * " ,coeff.iloc[2][1], "+ ",
      coeff.iloc[3][0], " * " ,coeff.iloc[3][1], "+ ",
      coeff.iloc[4][0], " * " ,coeff.iloc[4][1], "+ ",
      coeff.iloc[5][0], " * " ,coeff.iloc[5][1], "+ ",
      coeff.iloc[6][0], " * " ,coeff.iloc[6][1], "+ ",
      coeff.iloc[7][0], " * " ,coeff.iloc[7][1], "+ ",
      coeff.iloc[8][0], " * " ,coeff.iloc[8][1], "+ ",
      coeff.iloc[9][0], " * " ,coeff.iloc[9][1], "+ ",
      coeff.iloc[10][0], " * " ,coeff.iloc[10][1], "+ ",
      coeff.iloc[11][0], " * " ,coeff.iloc[11][1], "+ ",
      coeff.iloc[12][0], " * " ,coeff.iloc[12][1]) 